In [1]:
import sys, os, time
import random
import numpy as np
seed = 10
RNG = random.Random(seed)
random.seed(seed)
np.random.seed(seed)

import util as util
from util import adict, idict, xset, collate, load, render_dict, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
from itertools import chain, product
from tnt_units import load_unit_rules
import tnt_setup as setup

from flask_app import *

print(ping())

# out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Backend active: use "init" to init game


In [2]:
out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Set seed 10
Beginning phase: Setup
Game saved
adict(Axis:[9]{('Germany', [8]{([7]{'Munich'}, [4]{'Fortress', 'Tank', 'Infantry', 'AirForce'}), ([6]{'Ruhr', 'Berlin', 'Konigsberg'}, [1]{'Tank', 'Infantry', 'Carrier', 'Fleet', 'Submarine', 'Fortress', 'AirForce'})}), ('Italy', [16]{([15]{'Milan', 'Rome', 'Tripoli'}, [10]{'Tank', 'Infantry', 'Carrier', 'Fleet', 'Submarine', 'Fortress', 'AirForce'})})}, USSR:[25]{('USSR', [24]{([22]{'Urals', 'Kiev', 'Stalingrad', 'Moscow'}, [20]{'Fortress', 'Tank', 'Infantry', 'AirForce'}), ([23]{'Odessa', 'Kharkov', 'Leningrad', 'Baku'}, [17]{'Tank', 'Infantry', 'Carrier', 'Fleet', 'Submarine', 'Fortress', 'AirForce'})})}, West:[33]{('France', [40]{([39]{'Marseille', 'Paris', 'Algiers'}, [34]{'Tank', 'Infantry', 'Carrier', 'Fleet', 'Submarine', 'Fortress', 'AirForce'})}), ('Britain', [32]{([31]{'Suez', 'Glasgow', 'London', 'Bombay', 'Delhi'}, [26]{'Tank', 'Infantry', 'Carrier', 'Fleet', 'Submarine', 'Fortress', 'AirForce'})})})


In [2]:
out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Set seed 10
Beginning phase: Setup
Game saved
adict(Axis:[9]{('Italy', [16]{([15]{'Rome', 'Milan', 'Tripoli'}, [10]{'Fortress', 'Submarine', 'Infantry', 'AirForce', 'Fleet', 'Tank', 'Carrier'})}), ('Germany', [8]{([7]{'Munich'}, [4]{'Fortress', 'Tank', 'Infantry', 'AirForce'}), ([6]{'Konigsberg', 'Berlin', 'Ruhr'}, [1]{'Fortress', 'Submarine', 'Infantry', 'AirForce', 'Fleet', 'Tank', 'Carrier'})})}, USSR:[25]{('USSR', [24]{([22]{'Kiev', 'Stalingrad', 'Moscow', 'Urals'}, [20]{'Fortress', 'Tank', 'Infantry', 'AirForce'}), ([23]{'Odessa', 'Kharkov', 'Baku', 'Leningrad'}, [17]{'Fortress', 'Submarine', 'Infantry', 'AirForce', 'Fleet', 'Tank', 'Carrier'})})}, West:[33]{('Britain', [32]{([31]{'Glasgow', 'Delhi', 'Bombay', 'Suez', 'London'}, [26]{'Fortress', 'Submarine', 'Infantry', 'AirForce', 'Fleet', 'Tank', 'Carrier'})}), ('France', [40]{([39]{'Algiers', 'Paris', 'Marseille'}, [34]{'Fortress', 'Submarine', 'Infantry', 'AirForce', 'Fleet', 'Tank', 'Carrier'})})})


In [2]:
G = get_G()
# render_dict(G)

In [3]:
now = out.actions#list(util.decode_actions(out.actions))

In [4]:
print(now)

{('Italy', {({'Milan', 'Rome', 'Tripoli'}, {'Tank', 'AirForce', 'Infantry', 'Fortress', 'Submarine', 'Fleet', 'Carrier'})}), ('Germany', {({'Ruhr', 'Berlin', 'Konigsberg'}, {'Tank', 'AirForce', 'Infantry', 'Fortress', 'Submarine', 'Fleet', 'Carrier'}), ({'Munich'}, {'Infantry', 'Fortress', 'AirForce', 'Tank'})})}


In [8]:
print(prev)

{('Germany', {({'Berlin', 'Ruhr', 'Konigsberg'}, {'Fortress', 'AirForce', 'Carrier', 'Fleet', 'Tank', 'Submarine', 'Infantry'}), ({'Munich'}, {'Fortress', 'AirForce', 'Tank', 'Infantry'})}), ('Italy', {({'Milan', 'Rome', 'Tripoli'}, {'Fortress', 'AirForce', 'Carrier', 'Fleet', 'Tank', 'Submarine', 'Infantry'})})}


In [7]:
if prev != str(now):
    print(now)
    print(prev)
prev = now

In [4]:
now

{('Germany', {({'Konigsberg', 'Ruhr', 'Berlin'}, {'Carrier', 'Fortress', 'Submarine', 'Infantry', 'Fleet', 'AirForce', 'Tank'}), ({'Munich'}, {'Tank', 'AirForce', 'Infantry', 'Fortress'})}), ('Italy', {({'Milan', 'Rome', 'Tripoli'}, {'Carrier', 'Fortress', 'Submarine', 'Infantry', 'Fleet', 'AirForce', 'Tank'})})}

In [72]:
if now != prev:
    print('failed')
prev = now

failed


In [59]:
for x, y in zip(now,prev):
    if x != y:
        print(x, y)
        print('failed')
prev = now

('Italy', 'Tripoli', 'Submarine') ('Germany', 'Munich', 'AirForce')
failed
('Germany', 'Munich', 'AirForce') ('Italy', 'Tripoli', 'Submarine')
failed
('Italy', 'Tripoli', 'Fleet') ('Germany', 'Berlin', 'AirForce')
failed
('Italy', 'Milan', 'Infantry') ('Germany', 'Ruhr', 'Carrier')
failed
('Germany', 'Ruhr', 'Carrier') ('Germany', 'Konigsberg', 'Fleet')
failed
('Germany', 'Berlin', 'AirForce') ('Italy', 'Tripoli', 'Fleet')
failed
('Germany', 'Konigsberg', 'Fleet') ('Italy', 'Milan', 'Infantry')
failed
('Italy', 'Tripoli', 'AirForce') ('Germany', 'Berlin', 'Fleet')
failed
('Germany', 'Berlin', 'Fleet') ('Italy', 'Tripoli', 'AirForce')
failed
('Germany', 'Munich', 'Infantry') ('Germany', 'Ruhr', 'Tank')
failed
('Germany', 'Ruhr', 'Tank') ('Germany', 'Munich', 'Infantry')
failed
('Italy', 'Milan', 'Carrier') ('Germany', 'Ruhr', 'Infantry')
failed
('Germany', 'Ruhr', 'Infantry') ('Italy', 'Milan', 'Carrier')
failed
